In [1]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 16.5 MB/s eta 0:00:00


In [2]:
# مكتبة البيئات
import gymnasium as gym
# مكتبة نمباي
import numpy as np
# مكتبة تنسر فلو للشبكات العصبية
import tensorflow as tf

In [3]:
# إنشاء البيئة
env = gym.make('CartPole-v1')

In [4]:
# عدد الأفعال
num_actions = env.action_space.n
# عدد قيم الحالات
num_observations= env.observation_space.shape[0]


In [5]:
# شكل دخل الشبكة العصبية
input_shape = (num_observations,)

In [6]:
# دالة مخصصة لتعريف نموذج شبكة عصبية
def create_dqn_model(input_shape, num_actions):
    # بناء شبكة عصبية تحتوي على طبقتين مخفيتين
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),  # طبقة الدخل
        tf.keras.layers.Dense(24, activation='relu'),  # الطبقة الأولى
        tf.keras.layers.Dense(24, activation='relu'),  # الطبقة الثانية
        # طبقة الخرج والتي عدد الخرج بعدد الأفعال
        tf.keras.layers.Dense(num_actions, activation='linear')
    ])
    return model

In [7]:
# الشبكة الرئيسية
dqn_agent = create_dqn_model(input_shape, num_actions)

# الشبكة الهدف
target_network = create_dqn_model(input_shape, num_actions)

In [8]:
# المعاملات
learning_rate = 0.001  # معدل التعلم
discount_factor = 0.95  # معامل الخصم
epsilon = 1.0  # معدل الاستكشاف
epsilon_decay = 0.9955  # تخفيض الاستكشاف
min_epsilon = 0.1  # الحد الأدنى لإبسيلون

batch_size = 64  # حجم الدفعة للتدريب

In [9]:
from collections import deque
# الحجم الأعظمي لذاكرة التجارب
buffer_size = 1000000
# تهيئة ذاكرة التجارب
replay_buffer = deque(maxlen=buffer_size)

In [10]:
# تعريف دالة الخسارة والمُحسن
# دالة الخسارة: الخطأ التربيعي المتوسط
loss_fn = tf.keras.losses.MeanSquaredError()
# المُحسن: آدم
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [11]:
# دالة لتحديث أوزان الشبكة الهدف بشكل دوري
def update_target_network():
    target_network.set_weights(dqn_agent.get_weights())

In [ ]:
# عدد الحلقات التدريبية
num_episodes = 2000
# بدء التدريب
for episode in range(num_episodes):
    # إعادة تهيئة البيئة وتهيئة الحالة
    state, info = env.reset()
    state = state[np.newaxis, :]  # إعادة تشكيل الحالة لتصبح (1, 4)
    done = False
    episode_reward = 0  # إجمالي المكافآت في الحلقة

    while not done:
        # اختيار الفعل باستخدام سياسة إبسيلون
        if np.random.rand() < epsilon:
            # الاستكشاف بشكل عشوائي
            action = env.action_space.sample()
        else:
            # توقع القيم من النموذج
            q_values = dqn_agent(state)
             # اختيار الفعل الذي يحقق أكبر قيمة
            action = np.argmax(q_values.numpy())

        # تنفيذ الفعل والحصول على الحالة التالية والمكافأة
        next_state, reward, terminated, truncated, _ = env.step(action)
        next_state = next_state[np.newaxis, :]  # إعادة تشكيل الحالة التالية لتصبح (1, 4)
        episode_reward += reward  # إضافة المكافأة إلى إجمالي المكافأة في الحلقة

        # تخزين التجربة في الذاكرة
        replay_buffer.append((state, action, reward, next_state, terminated or truncated))

        # إذا كانت الذاكرة تحتوي على تجارب كافية، نبدأ في تدريب النموذج
        if len(replay_buffer) > batch_size:
            # اختيار دفعة من التجارب العشوائية
            batch = np.random.choice(len(replay_buffer), batch_size, replace=False)

            # قوائم الحالات و الأفعال والمكافأت والحالات التالية والإتمام
            states_batch, actions_batch, rewards_batch, next_states_batch, done_flags_batch = zip(*[replay_buffer[idx] for idx in batch])
            # دمج القوائم في مصفوفات واحدة
            states_batch = np.vstack(states_batch)
            next_states_batch = np.vstack(next_states_batch)

            # حساب قيم الخرج من الشبكة الرئيسية
            target_q_values = dqn_agent(states_batch).numpy()
            # حساب القيم المستهدفة باستخدام الشبكة الهدف
            next_q_values = target_network(next_states_batch).numpy()
            # إيجاد أكبر القيم
            max_next_q_values = np.max(next_q_values, axis=-1)

            # استخدام صيغة بل مان لتحديث القيم
            for i, action in enumerate(actions_batch):
                target_q_values[i, action] = rewards_batch[i] + discount_factor * max_next_q_values[i] * (1 - done_flags_batch[i])

            # حساب الخسارة
            with tf.GradientTape() as tape:
                current_q_values = dqn_agent(states_batch)
                loss = loss_fn(current_q_values, target_q_values)

            # حساب التدرجات
            gradients = tape.gradient(loss, dqn_agent.trainable_variables)
            # استخدام المحسن لتحديث الأوزان
            optimizer.apply_gradients(zip(gradients, dqn_agent.trainable_variables))

        # الانتقال إلى الحالة التالية
        state = next_state

        # التحقق إذا كانت الحلقة قد انتهت
        if terminated or truncated:
            done = True

    # تقليل احتمال الاستكشاف مع مرور الوقت
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # تحديث أوزان الشبكة الهدف كل 10 حلقات
    if (episode + 1) % 10 == 0:
        update_target_network()

     # طباعة تقدم التدريب كل 100 حلقة
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode + 1}: Reward = {episode_reward}, Epsilon = {epsilon:.3f}")

Episode 100: Reward = 112.0, Epsilon = 0.637
Episode 200: Reward = 237.0, Epsilon = 0.406
Episode 300: Reward = 31.0, Epsilon = 0.258
Episode 400: Reward = 182.0, Epsilon = 0.165
Episode 500: Reward = 17.0, Epsilon = 0.105
Episode 600: Reward = 116.0, Epsilon = 0.100
Episode 700: Reward = 195.0, Epsilon = 0.100
Episode 800: Reward = 111.0, Epsilon = 0.100
Episode 900: Reward = 130.0, Epsilon = 0.100


In [ ]:
 # رفع الدرايف
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# مسار مجلد العمل
working_folder='/content/drive/MyDrive/RLModels/'
# مسار النموذج
model_path = working_folder + "cartpole_dqn_model.h5"

In [ ]:
# حفظ النموذج المدرب
dqn_agent.save(model_path)